In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def find_best_answer(question, df):
    """最も類似度が高い答えを見つける関数"""
    # QAセットの質問文のリストを作成
    questions = df['Question'].tolist()
    
    # tf-idfベクトル化
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(questions)
    
    # テストデータのQをベクトル化
    question_vector = vectorizer.transform([question])
    
    # 類似度を計算
    cosine_similarities = linear_kernel(question_vector, tfidf_matrix).flatten()
    
    # 最も類似度が高いインデックスを取得
    best_match_idx = cosine_similarities.argmax()
    
    # 最も類似度が高い答えを返す
    return df.iloc[best_match_idx]['Answer']

# データフレーム型のQAセットの例
data = {
    'Question': ["天気は?", "今日の天気は?"],
    'Answer': ["晴れです。", "曇りです。"]
}
df = pd.DataFrame(data)

# テストデータの質問
test_question = "明日の天気は?"

# 最も類似度が高い答えを見つける
answer = find_best_answer(test_question, df)
print(answer)


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

def find_best_answer(question, df):
    """最も類似度が高い答えを見つける関数"""
    # sentence BERTモデルのロード
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    
    # QAセットの質問文のリストを作成
    questions = df['Question'].tolist()
    
    # QAセットの質問文をベクトル化
    question_embeddings = model.encode(questions, convert_to_tensor=True)
    
    # テストデータのQをベクトル化
    question_vector = model.encode(question, convert_to_tensor=True)
    
    # 類似度を計算
    cosine_scores = util.pytorch_cos_sim(question_vector, question_embeddings)[0]
    
    # 最も類似度が高いインデックスを取得
    best_match_idx = cosine_scores.argmax()
    
    # 最も類似度が高い答えを返す
    return df.iloc[best_match_idx]['Answer']

# データフレーム型のQAセットの例
data = {
    'Question': ["天気は?", "今日の天気は?"],
    'Answer': ["晴れです。", "曇りです。"]
}
df = pd.DataFrame(data)

# テストデータの質問
test_question = "明日の天気は?"

# 最も類似度が高い答えを見つける
answer = find_best_answer(test_question, df)
print(answer)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel

# BERTのトークナイザとモデルの初期化
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_embedding(text):
    """テキストをBERTで埋め込みベクトルに変換する関数"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

def find_best_answer(question, qa_set):
    """最も類似度が高い答えを見つける関数"""
    question_embedding = get_embedding(question)
    max_similarity = -1
    best_answer = ""
    
    for q, a in qa_set:
        q_embedding = get_embedding(q)
        similarity = cosine_similarity(question_embedding, q_embedding)
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = a
            
    return best_answer

# QAセットの例
qa_set = [("天気は?", "晴れです。"), ("今日の天気は?", "曇りです。")]

# テストデータの質問
test_question = "天気?"

# 最も類似度が高い答えを見つける
answer = find_best_answer(test_question, qa_set)
print(answer)


In [ ]:
import openai
from sklearn.metrics.pairwise import cosine_similarity

# OpenAIのAPIキーを設定
openai.api_key = 'YOUR_OPENAI_API_KEY'

def enhance_question_with_chatgpt(original_question):
    """ChatGPTを使用して質問を強化する関数"""
    response = openai.Completion.create(
      model="text-davinci-002",
      prompt=original_question,
      max_tokens=50
    )
    enhanced_question = original_question + " " + response.choices[0].text.strip()
    return enhanced_question

def find_best_answer(question, qa_set, embeddings):
    """最も類似度が高い答えを見つける関数"""
    question_embedding = get_embedding(question)  # あなたの埋め込み取得関数を使用
    max_similarity = -1
    best_answer = ""
    
    for idx, (q, a) in enumerate(qa_set):
        similarity = cosine_similarity([question_embedding], [embeddings[idx]])
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = a
            
    return best_answer

# QAセットの例
qa_set = [("天気は?", "晴れです。"), ("今日の天気は?", "曇りです。")]

# QAセットの質問の埋め込みを事前に計算
qa_embeddings = [get_embedding(q) for q, _ in qa_set]

# テストデータの質問
test_question = "天気?"

# 質問をChatGPTで強化
enhanced_test_question = enhance_question_with_chatgpt(test_question)

# 最も類似度が高い答えを見つける
answer = find_best_answer(enhanced_test_question, qa_set, qa_embeddings)
print(answer)


In [ ]:
import openai
import pandas as pd

# OpenAIのAPIキーを設定
openai.api_key = 'YOUR_OPENAI_API_KEY'

# 仮のQAペアのデータフレームを作成
data = {
    "question": ["製品の保証期間はどれくらいですか？", "製品の色は何色ありますか？"],
    "answer": ["製品の保証期間は1年間です。", "製品は赤、青、緑の3色展開です。"]
}
df_qa_pairs = pd.DataFrame(data)

def generate_clarifying_question(df):
    # 複数のQAペアを元に、関連性の高い質問を生成するためのプロンプトを作成
    prompt = "Given the following QA pairs:\n"
    for index, row in df.iterrows():
        prompt += f"Q: {row['question']}\nA: {row['answer']}\n"
    prompt += "What question can you ask to clarify which QA pair the user is referring to?"

    # ChatGPTに質問を生成させる
    response = openai.Completion.create(
      prompt=prompt,
      max_tokens=100
    )
    return response.choices[0].text.strip()

# 質問を生成
clarifying_question = generate_clarifying_question(df_qa_pairs)
print(f"Clarifying Question: {clarifying_question}")
